#### What I have done after completing task4_analysis_and_preprocess.ipynb

- I tried to make more data(like fields without erosion) from jp2 file using GIMP and laso tool(after 50 labeled photos I was totaly exhausted)

- Also i created csv for labeled image

- I have cut a few bigger png images from jp2 tile(why? I will explain below)


### How do I think solution should look like
First of all, I thought that I can pass jp2 map as input and binary map as output. Why have I abandoned that. I think that 10980x10980 input and output is impossible to compute with my pc or Google Colab cloud. What is more, 1 picture is too small dataset

Secondly, I thougth that it may be possible to divide huge map into small batches(parts) for example 2400 batches with size 224x224 but... I have abandoned it too.
So I have taken pictures I preprocessed, pick MobileNetV2 architectures(VGG16 detected cities perfectly but I need soil erosion!), pass it into RCNN

To be honest, I think that second one was the best. If only I had more time and more data...

So... I have chosen third one. I have cut in GIMP some parts of jp2 tile(approximately 500x500 px, you can see them in 'inputs' directory) like input and I wanted to have the same picture as output but with multiple bounding boxes. To be honest, I think that colab is more convenient so I will provide You with some photos that I got there

It would be great if I had more time to research and more time to tackle this problem

In [ ]:
import os

ORIG_BASE_PATH = "soils"
ORIG_IMAGES = os.path.sep.join([ORIG_BASE_PATH, "images"])
ORIG_ANNOTS = os.path.sep.join([ORIG_BASE_PATH, "erosion"])

BASE_PATH = "dataset"
POSITVE_PATH = os.path.sep.join([BASE_PATH, "positive"])
NEGATIVE_PATH = os.path.sep.join([BASE_PATH, "negative"])

MAX_PROPOSALS = 2000
MAX_PROPOSALS_INFER = 200

MAX_POSITIVE = 30
MAX_NEGATIVE = 10


INPUT_DIMS = (224, 224)

MODEL_PATH = "erosion_detector.h5"
ENCODER_PATH = "label_encoder.pickle"

MIN_PROBA = 0.995

In [ ]:
def compute_iou(boxA, boxB):
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

	iou = interArea / float(boxAArea + boxBArea - interArea)
	return iou

In [ ]:
import numpy as np
def non_max_suppression(boxes, overlapThresh):
    if len(boxes) == 0:
        return []
    pick = []
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        suppress = [last]
        for pos in range(0, last):
            j = idxs[pos]
            xx1 = max(x1[i], x1[j])
            yy1 = max(y1[i], y1[j])
            xx2 = min(x2[i], x2[j])
            yy2 = min(y2[i], y2[j])
            w = max(0, xx2 - xx1 + 1)
            h = max(0, yy2 - yy1 + 1)
          
            overlap = float(w * h) / area[j]
          
            if np.any(overlap > overlapThresh):
                suppress.append(pos)
            
        idxs = np.delete(idxs, suppress)
    return boxes[pick]

In [ ]:
!python3 fine_tune_rcnn.py

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import imutils
import pickle
import cv2
def predict(image_path, save_name="default.png"):
  print("[INFO] loading model and label binarizer...")
  model = load_model(MODEL_PATH)
  lb = pickle.loads(open(ENCODER_PATH, "rb").read())
  image = cv2.imread(image_path)
  image = imutils.resize(image, width=500)
  print("[INFO] running selective search...")
  ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
  ss.setBaseImage(image)
  ss.switchToSelectiveSearchFast()
  rects = ss.process()

  proposals = []
  boxes = []
  
  for (x, y, w, h) in rects[:MAX_PROPOSALS_INFER]:
      roi = image[y:y + h, x:x + w]
      roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
      roi = cv2.resize(roi, INPUT_DIMS,
                      interpolation=cv2.INTER_CUBIC)
      roi = img_to_array(roi)
      roi = preprocess_input(roi)
      proposals.append(roi)
      boxes.append((x, y, x + w, y + h))

  proposals = np.array(proposals, dtype="float32")
  boxes = np.array(boxes, dtype="int32")
  print("[INFO] proposal shape: {}".format(proposals.shape))
  print("[INFO] classifying proposals...")
  proba = model.predict(proposals)
  print("[INFO] applying NMS...")
  labels = lb.classes_[np.argmax(proba, axis=1)]
  idxs = np.where(labels == "positive")[0]
  boxes = boxes[idxs]
  proba = proba[idxs][:, 1]
  idxs = np.where(proba >= MIN_PROBA)
  boxes = boxes[idxs]
  proba = proba[idxs]
  # from google.colab.patches import cv2_imshow
  # import matplotlib.pyplot as plt
  # from PIL import Image

  clone = image.copy()
  # cv2_imshow(clone)
  clone = img_to_array(clone)

  for (box, prob) in zip(boxes, proba):
      (startX, startY, endX, endY) = box
      cv2.rectangle(clone, (startX, startY), (endX, endY),
                            (0, 255, 0), 2)
      y = startY - 10 if startY - 10 > 10 else startY + 10
      text = "Erosion: {:.2f}%".format(prob * 100)
      cv2.putText(clone, text, (startX, y),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)

  # cv2_imshow(clone)
  print()
  boxIdxs = non_max_suppression(boxes, proba)
  for i in boxIdxs:
      (startX, startY, endX, endY) = i
      image = cv2.rectangle(image, (startX, startY), (endX, endY),
                    (0, 255, 0), 2)
      y = startY - 10 if startY - 10 > 10 else startY + 10
      text = "Erosion"
      image =cv2.putText(image, text, (startX, y),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
  outfolder = "output"
  os.makedirs(outfolder, exist_ok=True)
  img_path = os.path.join(outfolder, str(save_name))
  img_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  cv2.imwrite(img_path, img_image)


In [ ]:
for i in range(1,9):
  print(str(i)+".png")
  predict("inputs/"+str(i)+".png", str(i)+".png")